In [1]:
# clone the ADIS repository
!git clone https://github.com/sathishkumar67/ADIS.git
# move the files to the current directory
!mv /kaggle/working/ADIS/* /kaggle/working/
# upgrade pip
!pip install --upgrade pip
# install the required packages
!pip install -r requirements.txt

Cloning into 'ADIS'...
remote: Enumerating objects: 1154, done.
remote: Total 1154 (delta 0), reused 0 (delta 0), pack-reused 1154 (from 2)
Receiving objects: 100% (1154/1154), 40.00 MiB | 20.55 MiB/s, done.
Resolving deltas: 100% (593/593), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.0/950.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [2]:
# necessary imports
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Define the global variables
REPO_ID = "pt-sk/ADIS" 
DATASET_NAME = "balanced_dataset" # dataset
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
LOCAL_DIR = os.getcwd()
TRAIN_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/train"
VAL_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/val"
TEST_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10                                               
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake'] 
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
MODEL_PATH = "yolo11n.pt" # model

In [4]:
# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)

os.remove(f"{LOCAL_DIR}/{DATASET_NAME}.zip")
print("deleted dataset.zip")

# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"No of Cores Available: {num_cores}")

balanced_dataset.zip:   0%|          | 0.00/7.04G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 7.07G/7.07G [00:41<00:00, 168MB/s]


deleted dataset.zip
No of Cores Available: 4


In [5]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

data yaml file written!.............


In [6]:
# Define the objective function
def objective(trial):
    
    # Define callback to report intermediate results
    def on_train_epoch_end(score, epoch):
        trial.report(score, step=epoch)  
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end" : on_train_epoch_end
    }
    
    # Define hyperparameters using Optuna suggestions
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)
    lrf = trial.suggest_float("lrf", 0.1, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0.0, 0.9)
    momentum = trial.suggest_float("momentum", 0.8, 0.99)
    
    CONFIG_DICT = {
    "task": "detect",
    "mode": "train",
    "bohb": True,
    "custom_callbacks": callbacks,
    "data": DATA_YAML_FILE,
    "batch": 320, # batch size
    "imgsz": 320,
    "save": True,
    "device": 0,
    "workers": num_cores,
    "pretrained": True,
    "optimizer": "AdamW",
    "seed": 42,
    "epochs": 60, # epochs
    "warmup_epochs": 5, # warmup
    "patience": 5} # patience for early stopping

    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)
    
    # Return validation mAP as the objective value
    return model.score 

In [7]:
import joblib
NUM_TRIALS = 10

# load the study
# study = optuna.create_study(direction='maximize', 
#                             sampler=optuna.samplers.TPESampler(), 
#                             pruner=optuna.pruners.HyperbandPruner(),
#                             study_name="yolo11_tuning",
#                             load_if_exists=True)
study = joblib.load("/kaggle/input/yolov11n-study1/yolov11n_study1.pkl")

# Optimize with a callback to stop after NUM_TRIALS complete trials
study.optimize(
    objective,
    n_trials=NUM_TRIALS)

joblib.dump(study, "/kaggle/working/optuna_study.pkl")

100%|██████████| 5.35M/5.35M [00:00<00:00, 19.4MB/s]


New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=60, time=None, patience=5, batch=320, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

100%|██████████| 755k/755k [00:00<00:00, 4.09MB/s]
E0000 00:00:1745490910.592399      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745490910.644332      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  blocks.Conv                                  [3, 16, 3, 2]                 
  1                  -1  1      4672  blocks.Conv                                  [16, 32, 3, 2]                
  2                  -1  1      6640  blocks.C3k2                                  [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  blocks.Conv                                  [64, 64, 3, 2]                
  4                  -1  1     26080  blocks.C3k2                                  [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  blocks.Conv                                  [128, 128, 3, 2]              
  6                  -1  1     87040  blocks.C3k2                                  [128, 128, 1, True]           
  7                  -1  1    295424  blocks.Con

train: Scanning /kaggle/working/balanced_dataset/train... 18139 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18139/18139 [00:13<00:00, 1383.30it/s]


...

                   all       2390       3083       0.87      0.772       0.86      0.612
                   Cat        239        262      0.936      0.892      0.957      0.732
                Cattle        239        351      0.832       0.65      0.777      0.546
               Chicken        239        495      0.882      0.661      0.795      0.539
                  Deer        239        326      0.901       0.81      0.889      0.667
                   Dog        239        272      0.848      0.721      0.815      0.479
              Squirrel        239        245       0.91      0.767      0.874      0.621
                 Eagle        239        252        0.9      0.833      0.918      0.772
                  Goat        239        332      0.809      0.732      0.787      0.574
               Rodents        239        280      0.776      0.811      0.862       0.59
                 Snake        239        268      0.903       0.84      0.923      0.601
Per-class IoU and Acc

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train10


[I 2025-04-24 14:18:25,438] Trial 14 finished with value: 0.61061 and parameters: {'lr0': 0.0031048056400465296, 'lrf': 0.10291667306529968, 'weight_decay': 2.9593646053342503e-05, 'warmup_momentum': 0.24150304872772754, 'momentum': 0.9899706105300101}. Best is trial 14 with value: 0.61061.


['/kaggle/working/optuna_study.pkl']